In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load training and test datasets
df_train = pd.read_csv("data_training.csv", parse_dates=["Tanggal"])
df_test = pd.read_csv("data_test_feb_apr_2025.csv", parse_dates=["Tanggal"])

FileNotFoundError: [Errno 2] No such file or directory: 'data_test_feb_apr_2025.csv'

In [ ]:
df_train = df_train.sort_values(by=["produk_encoded", "Tanggal"]).dropna()
df_test = df_test.sort_values(by=["produk_encoded", "Tanggal"]).dropna()

feature_cols = [
    'total_revenue', 'total_revenue_roll_mean_3', 'total_revenue_roll_std_3',
    'total_revenue_roll_min_3', 'total_revenue_roll_max_3', 'total_revenue_roll_mean_7',
    'total_revenue_roll_std_7', 'total_revenue_roll_min_7', 'total_revenue_roll_max_7',
    'total_revenue_roll_mean_14', 'total_revenue_roll_std_14', 'total_revenue_roll_min_14',
    'total_revenue_roll_max_14', 'total_revenue_roll_mean_21', 'total_revenue_roll_std_21',
    'total_revenue_roll_min_21', 'total_revenue_roll_max_21', 'total_revenue_roll_mean_30',
    'total_revenue_roll_std_30', 'total_revenue_roll_min_30', 'total_revenue_roll_max_30',
    'produk_encoded'
]
target_col = 'target_revenue_1_month_ahead'

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

print(len(df_train), len(df_test))  # Debugging output to check lengths

X_train_scaled = scaler_X.fit_transform(df_train[feature_cols])
X_test_scaled = scaler_X.transform(df_test[feature_cols])

y_train_scaled = scaler_y.fit_transform(df_train[[target_col]])
y_test_scaled = scaler_y.transform(df_test[[target_col]])

def create_sequences(X, y, produk_ids, sequence_length=30):
    Xs, ys = [], []
    for pid in np.unique(produk_ids):
        indices = np.where(produk_ids == pid)[0]
        X_p, y_p = X[indices], y[indices]
        for i in range(sequence_length, len(X_p)):
            Xs.append(X_p[i-sequence_length:i])
            ys.append(y_p[i])
    return np.array(Xs), np.array(ys)

# Produk IDs for mapping
produk_ids_train = df_train['produk_encoded'].values
produk_ids_test = df_test['produk_encoded'].values

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, produk_ids_train)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, produk_ids_test)

print(X_test_seq.shape, y_test_seq.shape)  # Debugging output to check shapes

input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])


model = Sequential([
    LSTM(128, activation='tanh', return_sequences=True, input_shape=input_shape),
    Dropout(0.3),
    LSTM(64, activation='tanh'),
    Dropout(0.3),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_test_seq, y_test_seq),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

y_pred_scaled = model.predict(X_test_seq)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_actual = scaler_y.inverse_transform(y_test_seq)

1578 116
(0,) (0,)


d:\Skripsi\Model Steps\Preprocessing\Step 1\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
22/23 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0192 - mae: 0.0812

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(64,), dtype=float32). Expected shape (None, 30, 22), but input has incompatible shape (64,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(64,), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>